In [ ]:
!pip install h5py

In [1]:
from scipy.io import loadmat
import numpy as np
import torch
import matplotlib.pyplot as plt

In [2]:
import h5py

def load_h5py_mat_file(filepath):
    data = {}
    with h5py.File(filepath, 'r') as file:
        def recursively_load_datasets(h5py_group, data_dict):
            for key, item in h5py_group.items():
                if isinstance(item, h5py.Dataset):  # If item is a dataset
                    data_dict[key] = item[()]  # Read the data
                elif isinstance(item, h5py.Group):  # If item is a group, read recursively
                    data_dict[key] = {}
                    recursively_load_datasets(item, data_dict[key])

        recursively_load_datasets(file, data)
    return data


In [3]:
def shape(data, indent=0):
    """
    Recursively prints the structure of the data dictionary.
    """
    for key, value in data.items():
        # Print the key with indentation
        print("  " * indent + f"{key}: ", end="")
        
        # Check the type of the item
        if isinstance(value, dict):  # Nested group
            print("Group")
            print_structure(value, indent + 1)
        elif isinstance(value, (list, tuple)):  # List or tuple
            print(f"List/Tuple of length {len(value)}")
        elif hasattr(value, 'shape'):  # Numpy arrays or datasets with a shape
            print(f"Dataset, shape={value.shape}")
        else:
            print("Unknown type")

In [26]:
data1 = load_h5py_mat_file('data/2D_ns_V1e-3_N5000_T50.mat')

In [28]:
shape(data1)

a: Dataset, shape=(64, 64, 5000)
t: Dataset, shape=(50, 1)
u: Dataset, shape=(50, 64, 64, 5000)


In [30]:
import scipy
class MatReader(object):
    def __init__(self, file_path, to_torch=True, to_cuda=False, to_float=True):
        super(MatReader, self).__init__()

        self.to_torch = to_torch
        self.to_cuda = to_cuda
        self.to_float = to_float

        self.file_path = file_path

        self.data = None
        self.old_mat = None
        self._load_file()

    def _load_file(self):
        try:
            self.data = scipy.io.loadmat(self.file_path)
            self.old_mat = True
        except:
            # self.data = h5py.File(self.file_path)
            self.old_mat = False

    def load_file(self, file_path):
        self.file_path = file_path
        self._load_file()

    def read_field(self, field):
        x = self.data[field]

        if not self.old_mat:
            x = x[()]
            x = np.transpose(x, axes=range(len(x.shape) - 1, -1, -1))

        if self.to_float:
            x = x.astype(np.float32)

        if self.to_torch:
            x = torch.from_numpy(x)

            if self.to_cuda:
                x = x.cuda()

        return x

    def set_cuda(self, to_cuda):
        self.to_cuda = to_cuda

    def set_torch(self, to_torch):
        self.to_torch = to_torch

    def set_float(self, to_float):
        self.to_float = to_float
    def list_keys(self):
        """List all keys available in the loaded .mat file."""
        if self.data is not None:
            return self.data.keys()
        else:
            raise ValueError("No data loaded. Please check the file path or file format.")

In [41]:
path = "data/2D_ns_V1e-3_N5000_T50_1.mat"
path2 = "data/1D_Lorenz_rho10.mat"

data = MatReader(path)
# data2 = MatReader(path2)
# data3 = MatReader('data/1D_burgers_data_R10.mat')

In [42]:
data.list_keys()

dict_keys(['__header__', '__version__', '__globals__', 'u'])

In [5]:
data2 = loadmat('data/1D_burgers_data_R10.mat')

In [17]:
for key, value in data2.items():
    if isinstance(value, np.ndarray):  # Check if the variable is an array
        print(f"Variable '{key}': Shape = {value.shape}")
    else:
        print(f"Variable '{key}' is not an array; shape is not applicable")


AttributeError: 'MatReader' object has no attribute 'items'